In [7]:
from IPython.display import display
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import log_loss
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.neural_network import MLPClassifier

In [8]:
df = pd.read_csv('loan_stat542.zip', compression='zip')

In [9]:
tdf = pd.read_csv('Project3_test_id.csv').astype(int)

In [10]:
df = df.reset_index()

In [11]:
with pd.option_context('display.max_columns', None):
    display(df.head(4))

,index,id,loan_amnt,term,int_rate,installment,grade,sub_grade,emp_title,emp_length,home_ownership,annual_inc,verification_status,loan_status,purpose,title,zip_code,addr_state,dti,earliest_cr_line,fico_range_low,fico_range_high,open_acc,pub_rec,revol_bal,revol_util,total_acc,initial_list_status,application_type,mort_acc,pub_rec_bankruptcies
0,0,38098114,15000,60 months,12.39,336.64,C,C1,MANAGEMENT,10+ years,RENT,78000.0,Source Verified,Fully Paid,debt_consolidation,Debt consolidation,235xx,VA,12.03,Aug-1994,750,754,6,0,138008.0,29.0,17,w,Individual,0.0,0.0
1,1,36805548,10400,36 months,6.99,321.08,A,A3,Truck Driver Delivery Personel,8 years,MORTGAGE,58000.0,Not Verified,Charged Off,credit_card,Credit card refinancing,937xx,CA,14.92,Sep-1989,710,714,17,0,6133.0,31.6,36,w,Individual,1.0,0.0
2,2,37842129,21425,60 months,15.59,516.36,D,D1,Programming Analysis Supervisor,6 years,RENT,63800.0,Source Verified,Fully Paid,credit_card,Credit card refinancing,658xx,MO,18.49,Aug-2003,685,689,10,0,16374.0,76.2,35,w,Individual,0.0,0.0
3,3,37662224,7650,36 months,13.66,260.20,C,C3,Technical Specialist,< 1 year,RENT,50000.0,Source Verified,Charged Off,debt_consolidation,Debt consolidation,850xx,AZ,34.81,Aug-2002,685,689,11,0,16822.0,91.9,20,f,Individual,0.0,0.0


We need to create categories for *emp_title* and *title*.

Somehow we need to handle *zip_code* as well.

*earliest_cr_line* should be stripped from mohths.

In [12]:
CAT_COLS = ["term", "grade", "sub_grade", "emp_length", "home_ownership", 
            "verification_status", "purpose", "addr_state", 
            "initial_list_status", "application_type"]

In [13]:
df = df.drop(columns=["emp_title", "zip_code", "title", "earliest_cr_line"])

In [14]:
df = pd.get_dummies(df, columns=CAT_COLS)

In [15]:
df['loan_status'] = np.where(df['loan_status']=="Fully Paid", 1, 0)

In [16]:
df = df.fillna(df.mean())

In [17]:
testidx = df.id.isin(tdf.test3)
trainidx = ~df.id.isin(tdf.test3)

In [18]:
mu = df.mean()
std = df.std()
df = (df - mu) / std
min_val = np.abs(df.min().min())
df.loan_status = (df.loan_status * std.loan_status) + mu.loan_status 

In [19]:
dfTest = df[testidx]
dfTrain = df[trainidx]
X_train = dfTrain.loc[:, dfTrain.columns != 'loan_status'].values
y_train = dfTrain.loan_status.values
X_test = dfTest.loc[:, dfTest.columns != 'loan_status'].values
y_test = dfTest.loan_status.values

In [ ]:
clf = LogisticRegression(random_state=0, solver='lbfgs', 
                         multi_class='multinomial', n_jobs=-1).fit(X_train, y_train)

In [81]:
clf = GradientBoostingClassifier(random_state=0)

In [78]:
clf = MLPClassifier(hidden_layer_sizes=(25,), alpha=0.000001, activation='relu')

In [82]:
clf.fit(X_train, y_train)

GradientBoostingClassifier(criterion='friedman_mse', init=None,
              learning_rate=0.1, loss='deviance', max_depth=3,
              max_features=None, max_leaf_nodes=None,
              min_impurity_decrease=0.0, min_impurity_split=None,
              min_samples_leaf=1, min_samples_split=2,
              min_weight_fraction_leaf=0.0, n_estimators=100,
              presort='auto', random_state=0, subsample=1.0, verbose=0,
              warm_start=False)

In [83]:
log_loss(y_test, clf.predict_proba(X_test))

0.44637480094682097